In [1]:
import numpy as np
import pandas as pd

In [2]:
books_df = pd.read_csv("./Books.csv")
books_df.head()

C:\Users\raj\AppData\Local\Temp\ipykernel_17656\279917578.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv("./Books.csv")


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [3]:
user_df = pd.read_csv("./Users.csv")
user_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [4]:
ratings_df = pd.read_csv("./Ratings.csv")
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
interactions_df = ratings_df.join(user_df, on="User-ID", lsuffix="_")

In [6]:
interactions_df.drop(['User-ID_', 'Location', 'Age'], axis = 1, inplace = True)

In [7]:
interactions_df.head()

,ISBN,Book-Rating,User-ID
0,034545104X,0,276726
1,0155061224,5,276727
2,0446520802,0,276728
3,052165615X,3,276730
4,0521795028,6,276730


In [8]:
interactions_df.groupby("User-ID")['Book-Rating'].size().sort_values(ascending=False).reset_index()

,User-ID,Book-Rating
0,11677,13602
1,198712,7550
2,153663,6109
3,98392,5891
4,35860,5850
...,...,...
105278,118580,1
105279,118577,1
105280,118575,1
105281,118564,1


In [9]:
interactions_df.nunique()

ISBN           340556
Book-Rating        11
User-ID        105283
dtype: int64

## Distance Based Collaborative Filtering

In [10]:
# interactions_df.pivot(index = "User-ID", columns = "ISBN", values="Book-Rating")
# Does not work because the resultant matrix will be too big

In [13]:
active_users_interactions_df = interactions_df[interactions_df["User-ID"].map(interactions_df["User-ID"].value_counts() > 200)]
user_book_rating_pivot = active_users_interactions_df.pivot(index = "User-ID", columns = "ISBN", values="Book-Rating")
user_book_rating_pivot.head()

ISBN,0330299891,0375404120,0586045007,9022906116,9032803328,9044922564,9044922572,9044922718,9044923161,904492401X,...,UNGRANDHOMMED,X000000000,"YOUTELLEM,AND",ZR903CX0003,"\0432534220\""""","\2842053052\""""",b00005wz75,cn108465,cn113107,Ô½crosoft
User-ID,,,,,,,,,,,,,,,,,,,,,
255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3364,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
user_book_rating_pivot.shape
## Active Users: 899, Total Books: 207291

(899, 207291)

In [15]:
user_book_rating_pivot.fillna(0, inplace = True)
user_book_rating_pivot.head()

ISBN,0330299891,0375404120,0586045007,9022906116,9032803328,9044922564,9044922572,9044922718,9044923161,904492401X,...,UNGRANDHOMMED,X000000000,"YOUTELLEM,AND",ZR903CX0003,"\0432534220\""""","\2842053052\""""",b00005wz75,cn108465,cn113107,Ô½crosoft
User-ID,,,,,,,,,,,,,,,,,,,,,
255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [114]:
from sklearn.metrics.pairwise import cosine_similarity

In [115]:
distance_metrix = cosine_similarity(user_book_rating_pivot)
## similarity with each of the other users 
# distance_metrix[sample_user_index].shape

(899,)

In [116]:
## user_book_rating_pivot.index==sample_user_id --> Generates the Boolean array
## np.where Returns the index value where True

def get_recommendations(user_id = 2277):
    sample_user_index = np.where(user_book_rating_pivot.index == user_id)[0][0]
    ## User index and similarity 
    user_similarity_scores = list(enumerate(distance_metrix[sample_user_index]))
    user_similarity_scores.sort(key = lambda x : x[1], reverse=True)
    similar_users = [index for index, score in user_similarity_scores[1:6]]
    similar_users = user_book_rating_pivot.index[similar_users].to_list()

    already_watched_movies = set(active_users_interactions_df[active_users_interactions_df["User-ID"] == user_id]["ISBN"].to_list())
    suggestions_list = []
    for user in similar_users:
        user_df = active_users_interactions_df[~active_users_interactions_df["ISBN"].isin(already_watched_movies)]
        user_movies = user_df.sort_values(by = "Book-Rating", ascending=False)["ISBN"][:5].to_list()
        suggestions_list.extend(user_movies)

    return set(suggestions_list)

In [117]:
get_recommendations(3364)

{'0307133125', '0399143629', '044022392X', '07009704500', '1568650108'}

## Deep Learning Approach to Collaborative Filtering